In [95]:
!rm hawaii.sqlite clean_hawaii_measurements.csv clean_hawaii_stations.csv

rm: clean_hawaii_measurements.csv: No such file or directory
rm: clean_hawaii_stations.csv: No such file or directory


In [96]:
# Dependencies

# Pandas
import pandas as pd

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# # SQL Alchemy
# from sqlalchemy import create_engine
# from sqlalchemy import Column, Float, Integer, String
# from sqlalchemy.ext.declarative import declarative_base

# Base = declarative_base()

In [97]:
#Use Pandas to read in the clean measurement file as DataFrame
csv_path1 = "Resources/clean_hawaii_measurements.csv"
measurements_df = pd.read_csv(csv_path1)
measurements_df.head(10)

station        date  prcp  tobs
0  USC00519397  2010-01-01  0.08    65
1  USC00519397  2010-01-02  0.00    63
2  USC00519397  2010-01-03  0.00    74
3  USC00519397  2010-01-04  0.00    76
4  USC00519397  2010-01-07  0.06    70
5  USC00519397  2010-01-08  0.00    64
6  USC00519397  2010-01-09  0.00    68
7  USC00519397  2010-01-10  0.00    73
8  USC00519397  2010-01-11  0.01    64
9  USC00519397  2010-01-12  0.00    61

In [98]:
#Use Pandas to read in the clean stations file as DataFrame
csv_path2 = "Resources/clean_hawaii_stations.csv"
stations_df = pd.read_csv(csv_path2)
stations_df.head(10)

station                                    name  latitude  longitude  \
0  USC00519397                    WAIKIKI 717.2, HI US  21.27160 -157.81680   
1  USC00513117                    KANEOHE 838.1, HI US  21.42340 -157.80150   
2  USC00514830  KUALOA RANCH HEADQUARTERS 886.9, HI US  21.52130 -157.83740   
3  USC00517948                       PEARL CITY, HI US  21.39340 -157.97510   
4  USC00518838              UPPER WAHIAWA 874.3, HI US  21.49920 -158.01110   
5  USC00519523      WAIMANALO EXPERIMENTAL FARM, HI US  21.33556 -157.71139   
6  USC00519281                     WAIHEE 837.5, HI US  21.45167 -157.84889   
7  USC00511918       HONOLULU OBSERVATORY 702.2, HI US  21.31520 -157.99920   
8  USC00516128            MANOA LYON ARBO 785.2, HI US  21.33310 -157.80250   

   elevation  
0        3.0  
1       14.6  
2        7.0  
3       11.9  
4      306.6  
5       19.5  
6       32.9  
7        0.9  
8      152.4

In [99]:
#------------------------------
#DATABASE CREATION
#------------------------------
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

Base = declarative_base()

In [100]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine('sqlite:///hawaii.sqlite')

In [101]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [102]:
# Use `declarative_base` from SQLAlchemy to model the Measurements and Stations table as ORM classes
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
class Measurements(Base):
    __tablename__ = "measurements"
    station = Column(String(255), primary_key = True)
    date = Column(String(255), primary_key = True)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [103]:
class Stations(Base):
    __tablename__ = "stations"
    station = Column(String(255), primary_key = True)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [104]:
# Use `create_all` to create the measurements and stations tables in the database
Base.metadata.create_all(engine)

In [105]:
# Load the cleaned csv files into a pandas dataframe
new_measuements_df = pd.read_csv(csv_path1)
new_stations_df = pd.read_csv(csv_path2)

In [106]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well

data1 = new_measuements_df.to_dict(orient='records')
data2 = new_stations_df.to_dict(orient='records')

In [107]:
# data1 and data2 is just a list of dictionaries that represent each row of data
#Print data1:
print(data1[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [108]:
#Print data2
print(data2[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [109]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [110]:
# Save the reference to the `measurements` table as a variable called `measurements_table`
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [111]:
# Save the reference to the `stations` table as a variable called `stations_table`
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [112]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [113]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(measurements_table.insert(), data1)
conn.execute(stations_table.insert(), data2)

In [114]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurements limit 5").fetchall()

[('USC00519397', '2010-01-01', 0.08, 65),
 ('USC00519397', '2010-01-02', 0.0, 63),
 ('USC00519397', '2010-01-03', 0.0, 74),
 ('USC00519397', '2010-01-04', 0.0, 76),
 ('USC00519397', '2010-01-07', 0.06, 70)]

In [115]:
conn.execute("select * from stations limit 5").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]